In [2]:
import pandas as pd

# Load the datasets
friends_positive = pd.read_csv('friends_positive.csv')
friends_negative = pd.read_csv('friends_negative.csv')
songs = pd.read_csv('songs.csv')

songs = songs.iloc[:, :-2]
songs.head()

,Song Name,Artists,Artist_ID,Track_ID,Happiness,Contentment,Confidence,Neutral,Sadness,Anger,...,Surprise,Disgust,Love,Excitement,Anticipation,Nostalgia,Confusion,Frustration,Longing,Optimism
0,Blinding Lights,The Weeknd,1Xyo4u8uXC1ZmMpatF05PJ,0VjIjW4GlUZAMYd2vXMi3b,1,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0
1,Memories,Maroon 5,04gDigrS5kc9YWfZHwBETP,2b8fOow8UzyDFAE27YhOZM,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,1,0
2,SAD!,XXXTENTACION,15UsOTVnJzReFVN1VCnxy4,3ee8Jmje8o58CHK66QrVC2,0,0,0,0,1,1,...,0,0,0,0,0,0,0,1,0,0
3,STAY (with Justin Bieber),"The Kid LAROI, Justin Bieber",2tIP7SsRs7vjIcLrU85W8J,5HCyWlXZPP0y6Gqq8TgA20,1,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0
4,Someone You Loved,Lewis Capaldi,4GNC7GD6oZMSxPGyXy4MNB,7qEHsqek33rTcFNT9PFqLf,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [3]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Extract emotion columns
emotion_columns = [
    'Happiness', 'Contentment', 'Confidence', 'Neutral', 'Sadness', 'Anger', 'Fear', 
    'Surprise', 'Disgust', 'Love', 'Excitement', 'Anticipation', 'Nostalgia', 'Confusion', 
    'Frustration', 'Longing', 'Optimism'
]

# Function to calculate cosine similarity between two emotion vectors
def get_cosine_similarity(dialogue_vector, song_vector):
    return cosine_similarity([dialogue_vector], [song_vector])[0][0]

# Extract emotion vectors for songs
songs_emotion_vectors = songs[emotion_columns].values

# Function to recommend songs based on emotions
def recommend_songs(dialogue_emotions):
    # Calculate the similarity between the dialogue emotions and each song
    similarities = [get_cosine_similarity(dialogue_emotions, song_vector) for song_vector in songs_emotion_vectors]
    
    # Get the indices of the top 3 most similar songs
    top_indices = np.argsort(similarities)[-3:][::-1]
    
    # Get the names of the top 3 songs
    recommended_songs = songs.iloc[top_indices]['Song Name'].values
    return ', '.join(recommended_songs)


In [4]:
# Function to process dataset and add song recommendations
def add_song_recommendations(friends_df):
    song_recommendations = []
    for index, row in friends_df.iterrows():
        # Extract emotion vector from the dialogue row
        dialogue_emotions = row[emotion_columns].values
        
        # Recommend songs based on the dialogue's emotions
        recommended_songs = recommend_songs(dialogue_emotions)
        song_recommendations.append(recommended_songs)
    
    # Add the song recommendations as a new column
    friends_df['Song Recommendations'] = song_recommendations
    return friends_df

# Apply the function to both datasets
friends_positive_with_songs = add_song_recommendations(friends_positive)
friends_negative_with_songs = add_song_recommendations(friends_negative)

# Save the updated datasets to CSV
friends_positive_with_songs.to_csv('friends_positive_with_songs.csv', index=False)
friends_negative_with_songs.to_csv('friends_negative_with_songs.csv', index=False)

print("Song recommendations added to the datasets and saved as 'friends_positive_with_songs.csv' and 'friends_negative_with_songs.csv'.")


Song recommendations added to the datasets and saved as 'friends_positive_with_songs.csv' and 'friends_negative_with_songs.csv'.
